# Complete Walkthrough

### Import necessary packages and load data

In [2]:
import math
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
# Load the data
data = pd.read_csv("data/background_color_experiment.csv")

# Print the first 10 rows
data.head(10)

,user_id,user_type,session_duration
0,BM3C0BJ7CS,variation,15.528769
1,MJWN6XNH6L,variation,32.287590
2,46ZPHHABLS,variation,43.718217
3,OHA298DHUG,variation,49.519702
4,AKJ77X6F4A,control,61.709028
5,BFNWMGU6DX,variation,71.779283
6,UFO2V8ZKFB,variation,23.291835
7,4CEIM3VRS9,control,25.219461
8,90AGF68FF8,control,26.240482
9,R3DQFO6068,variation,20.780244


The above data was adapated from Coursera course - [Probability & Statistics for Machine Learning & Data Science specialization](https://www.coursera.org/learn/machine-learning-probability-and-statistics/home/welcome) Week 4's coding assignment.

### Data exploration

Getting an overview of the dataframe using pandas `Dataframe.info()` method, which provides a concise column-wise summary, including the number of non-null entries and data types.

Another widely used overview methods in pandas is `df.describe()`, which provides a statistical summary of numerical columncs.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4186 entries, 0 to 4185
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           4186 non-null   object 
 1   user_type         4186 non-null   object 
 2   session_duration  4186 non-null   float64
dtypes: float64(1), object(2)
memory usage: 98.2+ KB


In [5]:
data.describe()

,session_duration
count,4186.000000
mean,33.378228
std,17.900628
min,5.644838
25%,20.997190
50%,29.390909
75%,40.712797
max,213.134368


In [6]:
data['user_type'].unique()

array(['variation', 'control'], dtype=object)

### Form hypothesis and revisit testing principles

This dataset provides a record of user's session duration over different website design (seperated by `variation` and `control` type).

In this context, the **null hypothesis** is that the change *did not* affect the time a visitor spend. Let's name $\mu_c$ is the average time a user in the **control group** spend in website, while $\mu_v$ is the average time a user in the **variation group** spend in website. With that, null hypothesis is then $H_0: \mu_c = \mu_v$ and the **alternative hypothesis** is $H_1:\mu_v > \mu_c \Leftrightarrow H_1: \mu_v - \mu_c > 0$.

This is a **right-tailed test**. 

As there are more than `2,000` users per group, it's reasonable to claim that the Central Limit Theorem that the average time for each group follows a **normal distribution**. (Note its the **group average time** follows a normal distribution, not each user's time spent). With the quantities defined as below:

- $\overline{X}_c$ - the control group **sample mean**.
- $\overline{X}_v$ - the variation group **sample mean**.
- $n_c$ - the control group **size**.
- $n_v$ - the variation group **size**.

We could express the CLT using symbols that:

- $$\overline{X}_c \sim N\left(\mu_c, \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$
- $$\overline{X}_v \sim N\left(\mu_v, \left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2\right)$$

With the assumptions of normality, $\overline{X}_v - \overline{X}_c$ also follows a normal distribution. So, if $H_0$ is true, then $\mu_c = \mu_v$ and $\mu_v - \mu_c = 0$, therefore:
 
$$\overline{X}_c - \overline{X}_v \sim N\left(\mu_v - \mu_c, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right) = N\left(0, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$

Or equivalently (with standardization):

$$\frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2}} \sim N(0, 1)$$

In [7]:
def get_stats(X):
    """
    Calculate basic statistics of a given data set. Extracted as a unique method to improve re-usability.

    Parameters:
    X (numpy.array): Input data.

    Returns:
    tuple: A tuple containing:
        - n (int): Number of elements in the data set.
        - x (float): Mean of the data set.
        - s (float): Sample standard deviation of the data set.
    """

    # Get the group size
    n = len(X)
    # Get the group mean
    x = np.mean(X)
    # Get the group sample standard deviation (do not forget to pass the parameter ddof if using the method .std)    
    # Note here `ddof` stands for the delta degree of freedom
    s = np.std(X, ddof=1)

    return (n,x,s)

In [8]:
# Separate the data from the two groups (sd stands for session duration)
control_sd_data = data[data["user_type"]=="control"]["session_duration"]
variation_sd_data = data[data["user_type"]=="variation"]["session_duration"]

# X_c stores the session tome for the control group and X_v, for the variation group. 
X_c = control_sd_data.to_numpy()
X_v = variation_sd_data.to_numpy()

# For the example dataset, we can have:
n_c, x_c, s_c = get_stats(X_c)
n_v, x_v, s_v = get_stats(X_v)

print(f"For X_c:\n\tn_c = {n_c}, x_c = {x_c:.2f}, s_c = {s_c:.2f} ")
print(f"For X_v:\n\tn_v = {n_v}, x_v = {x_v:.2f}, s_v = {s_v:.2f} ")

For X_c:
	n_c = 2069, x_c = 32.92, s_c = 17.54 
For X_v:
	n_v = 2117, x_v = 33.83, s_v = 18.24 


As we have **no access to the exact values** for $\sigma_v$ and $sigma_c$, we must replace the populartion variance with **sample standard deviation**, respectively, $s_c$ and $s_v$. Changing the population standard deviation to sample standard deviation results in the random variable to be changed from a **Normal Distribution** to a **t-student distribution**:
$$t = \frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{s_v}{\sqrt{n_v}}\right)^2 + \left(\frac{s_c}{\sqrt{n_c}}\right)^2}} \sim t_d$$

Where $d$ is the degree of freedom for this scenario.

$$d = \frac{\left[\frac{s_{v}^2}{n_v} + \frac{s_{c}^2}{n_c} \right]^2}{\frac{(s_{v}^2/n_v)^2}{n_v-1} + \frac{(s_{c}^2/n_c)^2}{n_c-1}}$$


In [9]:
def degrees_of_freedom(n_v, s_v, n_c, s_c):
    """
    Computes the degrees of freedom for two samples.

    Args:
        control_metrics (estimation_metrics_cont): The metrics for the control sample.
        variation_metrics (estimation_metrics_cont): The metrics for the variation sample.

    Returns:
        numpy.float: The degrees of freedom.
    """
    # Divide the numerator and the denominator calculation to make the code more readable.
    # Compute s_v^2/n_v 
    s_v_n_v = s_v**2 / n_v

    # Compute s_c^2/n_c 
    s_c_n_c = s_c**2 / n_c

    # Compute the numerator in the formula given above
    numerator = np.square(s_v_n_v + s_c_n_c)

    # Compute the denominator in the formula given above.
    denominator = np.square(s_c_n_c)/(n_c-1) + np.square(s_v_n_v)/(n_v-1)
    
    dof = numerator/denominator
        
    return dof

In [10]:
d = degrees_of_freedom(n_v, s_v, n_c, s_c)
print(f"The degrees of freedom for the t-student in this scenario is: {d:.2f}")

The degrees of freedom for the t-student in this scenario is: 4182.97


In [11]:
def t_value(n_v, x_v, s_v, n_c, x_c, s_c):
    """
    Computes the t-value with two samples.

    Args:
        control_metrics (estimation_metrics_cont): The metrics for the control sample.
        variation_metrics (estimation_metrics_cont): The metrics for the variation sample.

    Returns:
        numpy.float: The degrees of freedom.
    """

    # Compute s_v^2/n_v
    s_v_n_v = s_v**2 / n_v

    # Compute s_c^2/n_c 
    s_c_n_c = s_c**2 / n_c

    # Compute the numerator for the t-value as given in the formula above
    numerator = x_v - x_c

    # Compute the denominator for the t-value as given in the formula above. 
    denominator = np.sqrt(s_v_n_v + s_c_n_c)
    
    t = numerator/denominator

    return t

In [12]:
t = t_value(n_v, x_v, s_v, n_c, x_c, s_c)
print(f"The t-value for this experiment is: {t:.2f}")

The t-value for this experiment is: 1.64


With $d$ calculated and a selected significance level $\alpha$, we can perform the step of finding the p-value that:
$$p = P(t_d > t | H_0)$$

If this value is less than your significance level $\alpha$, then you **reject the null hypothesis**, because it means that you observed a value that is very unlikely to occur (unlikely here means that is less than the significance level you have set) if $H_0$ is true.

Note that $P(t_d \leq t)$ is the $\text{CDF}$ (cumulative distribution function) for the $t$-student distribution with $d$ degrees of freedom in the point $x = t$, so to compute $P(t_d > t)$ you may compute:

$$P(t_d > t) = 1 - \text{CDF}_{t_d}(t)$$

Since $P(t_d \leq t) + P(t_d > t) = 1$

In [13]:
def p_value(d, t_value):
    # Load the t-student distribution with $d$ degrees of freedom. 
    t_d = stats.t(df = d)

    # Compute the p-value, P(t_d > t). Remember to use the t_d.cdf with the proper adjustments as discussed above.
    p = 1 - t_d.cdf(t_value)

    return p

In [14]:
print(f"The p-value for t_15 with t-value = 1.10 is: {p_value(15, 1.10):.4f}")
print(f"The p-value for t_30 with t-value = 1.10 is: {p_value(30, 1.10):.4f}")

The p-value for t_15 with t-value = 1.10 is: 0.1443
The p-value for t_30 with t-value = 1.10 is: 0.1400


In [15]:
def make_decision(X_v, X_c, alpha = 0.05):
    """
    Combine all previous methods together
    """
    # Compute n_v, x_v and s_v
    n_v, x_v, s_v = get_stats(X_v)

    # Compute n_c, x_c and s_c
    n_c, x_c, s_c = get_stats(X_c)

    # Compute the degrees of freedom for the t-student distribution for this experiment.
    d = degrees_of_freedom(n_v, s_v, n_c, s_c)
    
    # Compute the t-value
    t = t_value(n_v, x_v, s_v, n_c, x_c, s_c)

    # Compute the p-value for the t-student distribution with d degrees of freedom
    p = p_value(d, t)

    # This is the decision step. Compare p with alpha to decide about rejecting H_0 or not. 

    if p < alpha:
        return 'Reject H_0'
    else:
        return 'Do not reject H_0'

In [16]:
alphas = [0.06, 0.05, 0.04, 0.01]
for alpha in alphas:
    print(f"For an alpha of {alpha} the decision is to: {make_decision(X_v, X_c, alpha = alpha)}")

For an alpha of 0.06 the decision is to: Reject H_0
For an alpha of 0.05 the decision is to: Do not reject H_0
For an alpha of 0.04 the decision is to: Do not reject H_0
For an alpha of 0.01 the decision is to: Do not reject H_0
